使用flyai运用TextCNN进行文本分类

定义参数

vocab_size = 20655      # 总词汇量
embedding_dim = 64      # 嵌入层大小
hidden_dim = 128        # Dense层大小
max_seq_len = 34        # 最大句长
num_filters = 128       # 卷积核数目
kernel_size = 5         # 卷积核尺寸
learning_rate = 1e-3    # 学习率
numclass = 3            # 类别数
filter_sizes = [3, 4, 5]

因为flyai平台的数据只能在flyai的平台上使用，在个人电脑无法使使用，故这里没有使用代码

定义卷积层和池化层

 pool_output = []
    for i, filter_size in enumerate(filter_sizes):
        shape = [filter_size, embedding_dim, 1, num_filters]
        W = tf.Variable(tf.truncated_normal(shape, 0.1), dtype=tf.float32)
        b = tf.Variable(tf.constant(0.1, shape=[num_filters]), dtype=tf.float32)
        conv = tf.nn.conv2d(x_input_embedded, W, padding='VALID', strides=[1, 1, 1, 1])
        h = tf.nn.relu(tf.nn.bias_add(conv, b), )
        max_pool = tf.nn.max_pool(h, ksize=[1, max_seq_len - filter_size + 1, 1, 1], strides=[1, 1, 1, 1], padding='VALID')
        pool_output.append(max_pool)

    num_filters_total = num_filters * len(filter_sizes)
    print(pool_output)
    pool = tf.concat(pool_output, 3)
    print(pool)
    pool_flat = tf.reshape(pool, [-1, num_filters_total])

    drop_out = tf.nn.dropout(pool_flat, keep_prob=keep_prob)

    W = tf.get_variable('W', shape=[num_filters_total, numclass], initializer=tf.contrib.layers.xavier_initializer())

    b = tf.Variable(tf.constant(0.1, shape=[numclass]))
    logits = tf.nn.xw_plus_b(drop_out, W, b)

获得accuary和运行






with tf.name_scope("score"):
    # 全连接层，后面接dropout以及relu激活
    # fc = tf.layers.dense(pooling, hidden_dim, name='fc1')
    # fc = tf.contrib.layers.dropout(fc, keep_prob)
    # fc = tf.nn.relu(fc)
    #
    # # 分类器
    # logits = tf.layers.dense(fc, numclass, name='fc2')
    y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1, name='y_pred')  # 预测类别

with tf.name_scope("optimize"):
    # 将label进行onehot转化
    one_hot_labels = tf.one_hot(input_y, depth=numclass, dtype=tf.float32)
    # 损失函数，交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
    loss = tf.reduce_mean(cross_entropy)
    # 优化器
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope("accuracy"):
    # 准确率
    correct_pred = tf.equal(tf.argmax(one_hot_labels, 1), y_pred_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='acc')

with tf.name_scope("summary"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary = tf.summary.merge_all()

best_score = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    # dataset.get_step() 获取数据的总迭代次数
    for step in range(dataset.get_step()):
        x_train, y_train = dataset.next_train_batch()

        a, b, c, d = dataset.get_all_data()
        print(a)
        # x_val, y_val = dataset.next_validation_batch()

        fetches = [loss, accuracy, train_op]
        feed_dict = {input_x: x_train, input_y: y_train, keep_prob: 0.9}
        loss_, accuracy_, _ = sess.run(fetches, feed_dict=feed_dict)

        summary = sess.run(merged_summary, feed_dict=feed_dict)
        train_writer.add_summary(summary, step)

        cur_step = str(step + 1) + "/" + str(dataset.get_step())
        print('The Current step per total: {} | The Current loss: {} | The Current ACC: {}'.
              format(cur_step, loss_, accuracy_))
        if step % 100 == 0:
            model.save_model(sess, MODEL_PATH, overwrite=True)

以下是运行的accu,可以看出效果不错

The Current step per total: 5/20 | The Current loss: 0.7554497122764587 | The Current ACC: 0.8125
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 6/20 | The Current loss: 0.7151859998703003 | The Current ACC: 0.765625
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 7/20 | The Current loss: 0.6841310858726501 | The Current ACC: 0.796875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 8/20 | The Current loss: 0.4181627035140991 | The Current ACC: 0.8125
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 9/20 | The Current loss: 0.3006260395050049 | The Current ACC: 0.859375
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 10/20 | The Current loss: 0.2413816750049591 | The Current ACC: 0.890625
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 11/20 | The Current loss: 0.25529056787490845 | The Current ACC: 0.890625
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 12/20 | The Current loss: 0.26477718353271484 | The Current ACC: 0.921875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 13/20 | The Current loss: 0.16862069070339203 | The Current ACC: 0.96875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 14/20 | The Current loss: 0.19627314805984497 | The Current ACC: 0.953125
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 15/20 | The Current loss: 0.14733761548995972 | The Current ACC: 0.96875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 16/20 | The Current loss: 0.11074429750442505 | The Current ACC: 0.984375
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 17/20 | The Current loss: 0.12784716486930847 | The Current ACC: 0.96875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 18/20 | The Current loss: 0.1121433675289154 | The Current ACC: 0.96875
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 19/20 | The Current loss: 0.06821130216121674 | The Current ACC: 1.0
[{'text': "i got bitches definition i'm with some female friends who don't know that i'm referring to them as bitches"}, {'text': 'i got all these hoes starin when i walk thru'}, {'text': 'the only thing better than a bargain is a yankee bargain'}, {'text': 'columbus found america and we killed some injuns sincethecubswonaworldseries'}, {'text': 'these hoes will fake like they really f ck with you but be hating on you lowkey'}, {'text': "you're no way near about that life bitch lmao"}, {'text': 'idk why your favoriting shit hoe bye'}, {'text': "them hoes come them hoes go it's a revolving cycle"}, {'text': 'but then again i aint never worried bout no bitch nor nigga'}, {'text': "you are a pussy without your guns bitch as spic your daddy doesn't even respect the sperm he drop in your moms stank pussy"}, {'text': 'birth control should be mandatory for some of you hoes'}, {'text': 'the jets are trash my bad i see what you meant'}, {'text': "i'm a vipperloid bitch p"}, {'text': 'lmfaoooo fizz baby mama snatched that bitch soul'}, {'text': 'god bless those delusional bitches still trying to keep duck face alive'}, {'text': 'on the northside wildbills with come up on a check left that hoe alone'}, {'text': "you can't play me for a fool my niggah"}, {'text': 'is it ghetto to grease yo scalp at work'}, {'text': 'bitch i texted you'}, {'text': "my bestfriend is the biggest bitch i know that's why we are best friends lol"}, {'text': 'the bitch loudly cheering for the netherlands at rosa mexicano does not seem to value her life'}, {'text': 'knowing i can picture that hoe in my head he good now knocked the wind out that boy'}, {'text': "if you ask me every bitch gay not me i'm all sexual"}, {'text': 'yeah but your the one who wanted followers you gotta work it bitch'}, {'text': 'nigga hungry then a bitch too'}, {'text': 'thick red bones in style or nah'}, {'text': 'dirty whore i love you but you a hoe but i love you jesus is watching'}, {'text': 'fat bitches twitter names be princesscookieneckbackcrack lmaoo'}, {'text': 'pussy ass schedule'}, {'text': "i'd get a girlfriend if i had hoes tho lmaoo"}, {'text': 'yuh bitch voz de mando at ok was the best mf friday yet'}, {'text': "good she's a ghetto thug so fpd just arrested for being in the street ferguson"}, {'text': 'den you enjoy da nite wit a colored man we do it again in weeks lets me no if i needs ta bring da magnums'}, {'text': 'here come wiggas soft tweets'}, {'text': 'lmaoooo jesus healed folks wit leoparcy but yall hoes wanna get leopard print tattoos'}, {'text': 'i knew farquaad a faggot'}, {'text': "niggas be frontin for some pussy they ain't wit me"}, {'text': 'pineapples on sale at publix hoes on sale too up stairs'}, {'text': 'i was raised the person that uses a gun and cant take an ass whipping is a pussy'}, {'text': "if you ate my pussy don't ever try to disrespect me fck nigga i fed you"}, {'text': 'when niggas wanna cheat they start an argument with their girl bitch i told you i hate ketchup on my hotdogs fuck this im out'}, {'text': 'slow the fuck down bitch i know where you live classy young lady'}, {'text': 'pit bulls photographed as lovely fairy tale creatures they are fairy tale creatures'}, {'text': 'ctfu bitch talm bout i needs my respect'}, {'text': 'girls go talk to your other hoes game too strong'}, {'text': 'pussy ass nigguh name jesus and his daddy'}, {'text': 'theses niggas hoes and these bitches greedy'}, {'text': "you're a fucking nigger"}, {'text': 'faggot'}, {'text': 'yeah right pussy'}, {'text': 'about to smack some peckerwoods in a judge pit'}, {'text': 'at least i have a boyfriend dumb bitch'}, {'text': "you're all niggers"}, {'text': 'if i ever ate your pussy before and you fucked over niggas before that let me know right now because you owe me you dirty bitch'}, {'text': 'i hate taking a shit at public places bc i feel like a faggot when im placing the toilet paper on the toilet seat'}, {'text': 'i been goin thru some shit from my niggas to my bitch they lookin like they wanna switch'}, {'text': "we hate the yankees though right i feel like i'm really good at hating them"}, {'text': "that nig darion couldn't help but act like the camera was on him he didn't keep any composure eattin those crawfish hahaha"}, {'text': 'i never put no hoes b my bros'}, {'text': 'yeah basic bitches are a thing but how many straight white men do you know who are all the same and so boring'}, {'text': 'kissin small lips is trash lol yessssss'}, {'text': 'where this hoe benga at'}, {'text': 'peter pan that ho never land that ho'}, {'text': 'as soon as we left iraq all these towel heads think they can fucking barge in and take over what we worked so hard to set up pisses me off'}, {'text': 'lol good now i got them nasty rednecks off my back i can take care of business he serious'}, {'text': "i said i don't like it it tastes like wolf pussy what how do you even know what that tastes like"}, {'text': "a real nigga ain't complete without a real bitch"}, {'text': 'is having a birthday so make sure you tell that bitch be happy amp celebrate without drugs'}, {'text': 'bitches like u be proving my point u speak on shit u know nothing about i got a fucking at studio movie grill just stfu'}, {'text': "bitches be half naked in their avi's talking about i don't send nudes"}, {'text': 'gstar suspenders got a bad bitch bitch'}, {'text': "i'm in this bitch yaaa i do my thanggg"}, {'text': 'bruh this bitch booty so flat that its muscular'}, {'text': 'bae gotta bitch whoops'}, {'text': 'after i fuck your bitch'}, {'text': "his defense is trash and he's been rocked by jesus soto karass and diego chaves"}, {'text': 'those guys are pussies'}, {'text': "tbh but a niggah can't have a normal convo with someone lmao hoes will be hoes brian lol"}, {'text': 'damn karlie redd now all these hoes done seen your man junk'}, {'text': 'that bitch want my numba she gotta fuck my bruda'}, {'text': "after sex a bitch be like so what are we and you like probably parents cause i ain't pull out"}, {'text': 'this bitch almost lost her life'}, {'text': 'reminds me of oreos lmao'}, {'text': "gabby's a bitch and has two accounts"}, {'text': 'i just read prolly tweets from different girls about how their looong relationship ended today daaaamn yo hoe will get slayed'}, {'text': "i'm an early bird and i'm a night owl so i'm wise and have worms"}, {'text': 'you always find something to bitch about'}, {'text': "ill finger ur tight pussy so slowly you'll be moaning my name bby"}, {'text': 'pudding it right in her pussy'}, {'text': 'we have a project that could lead into a military program but requires a full op mock up for testing'}, {'text': 'you bitches are grade a weird'}, {'text': 'amp thots are wearing uggs rt itsfallbecause negros are pulling out their timbs'}, {'text': "ladies do y'all shave wax before visiting ur obgyn or let it grow out because u don't wanna seem like a hoe"}, {'text': 'theres a girl in your avi you must get so many bitches'}, {'text': 'grudges are for weirdos and bitches'}, {'text': "but seriously it's the only reason i use a mega whatsoever pure favoritism also those sweet fuzzy wings"}, {'text': 'teabagger math'}, {'text': "these hoes ain't loyal n these niggas ain't either fuck ah bitch get ah bird"}, {'text': 'work out hoe'}, {'text': 'lol bitch'}]
The Current step per total: 20/20 | The Current loss: 0.05969906970858574 | The Current ACC: 1.0